# The Forced Rayleigh Oscillator #

## Nonresonant case ##

Support material for the book __Perturbation Methods using Backward Error__ by Robert M. Corless and Nicolas Fillion, in preparation.

Copyright (c) 2024 Robert M. Corless

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

$\newcommand{\e}{\varepsilon}$

This notebook explores the forced Rayleigh oscillator
\begin{equation}
\ddot{y} - \e \dot{y}\left( 1 - \frac{4}{3}\dot{y}^2 \right) + y = 2F\cos(\Omega t + \Phi)
\end{equation}
by perturbation expansion up to and including the $O(\e)$ terms, so it has a residual $O(\e^2)$.  We will encounter the so-called "small divisors" problem, and discover that we will have to do separate expansions for $\Omega \approx 1$ (called the _resonant_ case), $\Omega \approx 1/3$ (called the _superharmonic_ case), $\Omega \approx 3$ (called the _subharmonic_ case), and $\Omega$ none of those three (called the _nonresonant_ case).  We will do the nonresonant case first, and see how "small divisors" require us to consider the other three. 

## Nonresonant case ##

In [1]:
macro( e=varepsilon );
WeaklyNonlinear := diff(y(t), t, t) 
  - e*diff(y(t), t)*(1 - 4/3*diff(y(t), t)^2) + y(t) 
  - F*exp(I*(Omega*t+Phi)) - F*exp(-I*(Omega*t+Phi));

                                       e

$$\frac{d^{2}}{d t^{2}}y \left(t \right)-\varepsilon  \left(\frac{d}{d t}y \left(t \right)\right) \left(1-\frac{4 \left(\frac{d}{d t}y \left(t \right)\right)^{2}}{3}\right)+y \left(t \right)-F \,{\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}-F \,{\mathrm e}^{\mathrm{-I} \left(\Omega  t +\Phi \right)}$$

In [2]:
LinearODE := diff(y(t),t,t) + y(t);

$$\frac{d^{2}}{d t^{2}}y \left(t \right)+y \left(t \right)$$

In [3]:
sol := dsolve( eval( WeaklyNonlinear, e=0), y(t) );

kilobytes used=5428, alloc=13872, time=0.78

$$y \left(t \right) = \sin \left(t \right) c_{2} +\cos \left(t \right) c_{1} -\frac{2 F \cos \left(\Omega  t +\Phi \right)}{\Omega^{2}-1}$$

In [4]:
simplersol := eval( rhs(sol), [c__1=0, c__2=0 ] ) 
  + A*exp(I*phi)*exp(I*t) + A*exp(-I*phi)*exp(-I*t) ;

$$-\frac{2 F \cos \left(\Omega  t +\Phi \right)}{\Omega^{2}-1}+A \,{\mathrm e}^{\mathrm{I} \phi} {\mathrm e}^{\mathrm{I} t}+A \,{\mathrm e}^{\mathrm{-I} \phi} {\mathrm e}^{\mathrm{-I} t}$$

In [5]:
residual := convert( simplify(
  series( eval( WeaklyNonlinear, y(t)=simplersol ),e,2)
                              ), exp ):
residual := coeff( residual, e, 1 ):
residual := convert( residual, trig ): # dsolve likes trig
residual := combine( residual, trig );
residual := collect( residual, [sin,cos,F,A], 
  distributed, m->normal(m,expanded));
residual := collect( residual, [sin,cos] );

                        Omega  - 2 Omega  + 1

$$\left(-\frac{16 \Omega^{2} F^{2} A}{\Omega^{4}-2 \Omega^{2}+1}-8 A^{3}+2 A \right) \sin \left(\phi +t \right)+\frac{8 A^{3} \sin \left(3 \phi +3 t \right)}{3}+\left(\frac{8 \Omega^{3} F^{3}}{\Omega^{6}-3 \Omega^{4}+3 \Omega^{2}-1}+\frac{16 \Omega  F \,A^{2}}{\Omega^{2}-1}-\frac{2 F \Omega}{\Omega^{2}-1}\right) \sin \left(\Omega  t +\Phi \right)-\frac{8 \Omega^{3} F^{3} \sin \left(3 \Omega  t +3 \Phi \right)}{3 \Omega^{6}-9 \Omega^{4}+9 \Omega^{2}-3}-\frac{8 \Omega  \sin \left(\Omega  t +\Phi -2 \phi -2 t \right) F \,A^{2}}{\Omega^{2}-1}-\frac{8 \Omega  \sin \left(\Omega  t +\Phi +2 \phi +2 t \right) F \,A^{2}}{\Omega^{2}-1}-\frac{8 \Omega^{2} \sin \left(2 \Omega  t +2 \Phi -\phi -t \right) F^{2} A}{\Omega^{4}-2 \Omega^{2}+1}+\frac{8 \Omega^{2} \sin \left(2 \Omega  t +2 \Phi +\phi +t \right) F^{2} A}{\Omega^{4}-2 \Omega^{2}+1}$$

In [6]:
# SHOlveTerm --- solve a simple harmonic oscillator, harmonically forced
#
# Copyright 2024 Robert M. Corless 
#
# dsolve can be inefficient at solving y'' + omega^2*y = sum of simple terms
# (probably getting bogged down in type checking or simplification
#  or its own heuristics, or perhaps it's just its generality).
# This routine attempts to do this simpler task more efficiently,
# by giving up on generality.
#
# Input: expr  : right-hand side: polynomial in sine/cos/exp only
#                with coefficients polynomials in var
#        var   : variable (usually t)
#        omega : natural frequency
#
# Processing: map over a sum of terms
#             match the terms to poly(var)*exp( I*(freq*var + phase) )
#                            or  poly(var)*sin( freq*var + phase )
#                            or  poly(var)*cos( freq*var + phase )
#
#          solve the problem y'' + omega^2*y = sum poly(var)* fn( freq*var+phase )
#
# Output: a particular solution to y'' + omega^2 = ...
#         where the rhs is a sum of polynomials in var times sin/cos/exp.
#
# Comments: dsolve would solve this, but for large expressions might take
# a seemingly unreasonable length of time to do so, and may return 
# unevaluated integrals (because it thinks the answers are too complicated).
# However, this simple routine will not succeed for unexpected inputs.
# Take your pick.

SHOlveTerm := proc( expr, var, omega )
	local A, c, fn, freq, la, m, phase, sol, term, zerp;
	description "Solve y''+ omega^2*y=expr";
    if has( expr, sin ) or has( expr, cos ) then
      # convert,exp will make cos(A) --> (exp(I*A)+exp(-I*A))/2, etc
      # expand will expand (A*exp(B) + C*exp(D))*(E*exp(F)+G*exp(H))
      # to A*E*exp(B)*exp(F) + A*G*exp(B)*exp(H) + ... 
      # combine,exp will make exp(B)*exp(F)-->exp(B+F)
      term := combine( expand( convert(expr,exp ) ), exp );
    else
      term := expr;
    end if; 
	if type( term, `+` ) then
      # Apply this procedure to each term in a sum  
	  return map( procname, term, var, omega );
	else 
      # Look for A*exp(I*(freq*var+phase))
      term := combine( term, exp );
      A := eval(term, exp=1);
      fn := normal( term/A );
      # Each term *should* be A*exp(B).  Should be.
      if normal( expr-A*fn ) <> 0 then
        error "Unexpected term type", term;
      end if;
      # Identify frequency and phase
      if match( fn=exp(I*(freq*var+phase)), var, 'la' ) then
        if eval(freq,la)^2 = omega^2 then
          # resonant case
          m := degree(A, var) + 1; # A must be polynomial
        else
          m := degree(A, var);
        end if;
        P := add( c[k]*var^k, k=0..m );
        # y = P(var)*exp(I*(freq*var+phase))
        # for P(var) a polynomial to be found.
        zerp := eval( omega^2*P + diff(P,var,var) + 2*I*freq*diff(P,var) - freq^2*P - A, la);
        sol := solve( {seq( coeff(zerp,var,k), k=0..m)}, {seq(c[k],k=0..m)} );
        # triangular system, guaranteed to have a solution
        # Throw away the redundant terms c[0]*exp(I*(freq*var+phase))
        return eval( eval( eval(P,sol), c[0]=0)*exp(I*(freq*var+phase)), la );
      else
        error "Unexpected term type, after match", term, A, fn;
      end if;
    end if;
end proc:

We see our first "small divisor" already in the solution to the zeroth order equation.  When $\Omega^2 \approx 1$, the amplitude is not small.

In [7]:
firstcorrection := SHOlveTerm( -residual, t, 1 );

$$-\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(\left(-\Omega -2\right) t -\Phi -2 \phi \right)}}{\left(\Omega^{2}-1\right) \left(\Omega^{2}+4 \Omega +3\right)}-\frac{8 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega^{2}-1\right)^{2}}+\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(\left(\Omega -2\right) t +\Phi -2 \phi \right)}}{\left(\Omega^{2}-1\right) \left(\Omega^{2}-4 \Omega +3\right)}+\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(3 \Omega  t +3 \Phi \right)}}{3 \left(\Omega^{6}-3 \Omega^{4}+3 \Omega^{2}-1\right) \left(9 \Omega^{2}-1\right)}-\frac{\mathrm{I} A^{3} {\mathrm e}^{\mathrm{I} \left(3 \phi +3 t \right)}}{6}+\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{\mathrm{I} \left(\left(2 \Omega -1\right) t +2 \Phi -\phi \right)}}{\left(\Omega^{4}-2 \Omega^{2}+1\right) \left(\Omega -1\right)}+\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{\mathrm{I} \left(\left(-2 \Omega -1\right) t -2 \Phi -\phi \right)}}{\left(\Omega^{4}-2 \Omega^{2}+1\right) \left(\Omega +1\right)}-2 A^{3} t \,{\mathrm e}^{\mathrm{I} \left(-\phi -t \right)}+\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega^{6}-3 \Omega^{4}+3 \Omega^{2}-1\right) \left(\Omega^{2}-1\right)}-2 A^{3} t \,{\mathrm e}^{\mathrm{I} \left(\phi +t \right)}-\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(-3 \Omega  t -3 \Phi \right)}}{3 \left(\Omega^{6}-3 \Omega^{4}+3 \Omega^{2}-1\right) \left(9 \Omega^{2}-1\right)}-\frac{4 \Omega^{2} F^{2} A t \,{\mathrm e}^{\mathrm{I} \left(-\phi -t \right)}}{\Omega^{4}-2 \Omega^{2}+1}-\frac{4 \Omega^{2} F^{2} A t \,{\mathrm e}^{\mathrm{I} \left(\phi +t \right)}}{\Omega^{4}-2 \Omega^{2}+1}+\frac{A t \,{\mathrm e}^{\mathrm{I} \left(-\phi -t \right)}}{2}+\frac{8 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega^{2}-1\right)^{2}}-\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega^{6}-3 \Omega^{4}+3 \Omega^{2}-1\right) \left(\Omega^{2}-1\right)}-\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{\mathrm{I} \left(\left(-2 \Omega +1\right) t -2 \Phi +\phi \right)}}{\left(\Omega^{4}-2 \Omega^{2}+1\right) \left(\Omega -1\right)}-\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{\mathrm{I} \left(\left(2 \Omega +1\right) t +2 \Phi +\phi \right)}}{\left(\Omega^{4}-2 \Omega^{2}+1\right) \left(\Omega +1\right)}-\frac{\mathrm{I} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega^{2}-1\right)^{2}}+\frac{\mathrm{I} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega^{2}-1\right)^{2}}+\frac{\mathrm{I} A^{3} {\mathrm e}^{\mathrm{I} \left(-3 \phi -3 t \right)}}{6}+\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(\left(\Omega +2\right) t +\Phi +2 \phi \right)}}{\left(\Omega^{2}-1\right) \left(\Omega^{2}+4 \Omega +3\right)}+\frac{A t \,{\mathrm e}^{\mathrm{I} \left(\phi +t \right)}}{2}-\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \left(\left(-\Omega +2\right) t -\Phi +2 \phi \right)}}{\left(\Omega^{2}-1\right) \left(\Omega^{2}-4 \Omega +3\right)}$$

In [8]:
firstcorrection := collect( combine(expand(convert( (firstcorrection), exp )),exp), exp, factor );

$$-\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{-3 \,\mathrm{I} \Omega  t -3 \,\mathrm{I} \Phi}}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(3 \Omega -1\right) \left(3 \Omega +1\right)}-\frac{A t \left(4 A^{2} \Omega^{4}-8 A^{2} \Omega^{2}+8 F^{2} \Omega^{2}-\Omega^{4}+4 A^{2}+2 \Omega^{2}-1\right) {\mathrm e}^{-\mathrm{I} t -\mathrm{I} \phi}}{2 \left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-\frac{A t \left(4 A^{2} \Omega^{4}-8 A^{2} \Omega^{2}+8 F^{2} \Omega^{2}-\Omega^{4}+4 A^{2}+2 \Omega^{2}-1\right) {\mathrm e}^{\mathrm{I} t +\mathrm{I} \phi}}{2 \left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{-2 \,\mathrm{I} t \Omega -2 \,\mathrm{I} \Phi +\mathrm{I} \phi +\mathrm{I} t}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}-\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{-\mathrm{I} \Omega  t -\mathrm{I} \Phi +2 \,\mathrm{I} \phi +2 \,\mathrm{I} t}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right) \left(\Omega -3\right)}+\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \Omega  t +\mathrm{I} \Phi -2 \,\mathrm{I} \phi -2 \,\mathrm{I} t}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right) \left(\Omega -3\right)}-\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{2 \,\mathrm{I} \Omega  t +2 \,\mathrm{I} \Phi +\mathrm{I} \phi +\mathrm{I} t}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}-\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{-\mathrm{I} \Omega  t -\mathrm{I} \Phi -2 \,\mathrm{I} \phi -2 \,\mathrm{I} t}}{\left(\Omega -1\right) \left(\Omega +1\right)^{2} \left(\Omega +3\right)}-\frac{\mathrm{I} A^{3} {\mathrm e}^{3 \,\mathrm{I} \phi +3 \,\mathrm{I} t}}{6}-\frac{\mathrm{I} \left(8 A^{2} \Omega^{4}-16 A^{2} \Omega^{2}+4 F^{2} \Omega^{2}-\Omega^{4}+8 A^{2}+2 \Omega^{2}-1\right) F \Omega  \,{\mathrm e}^{\mathrm{I} \Omega  t +\mathrm{I} \Phi}}{\left(\Omega -1\right)^{4} \left(\Omega +1\right)^{4}}+\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{-2 \,\mathrm{I} t \Omega -2 \,\mathrm{I} \Phi -\mathrm{I} \phi -\mathrm{I} t}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}+\frac{\mathrm{I} \Omega  \,F^{2} A \,{\mathrm e}^{2 \,\mathrm{I} \Omega  t +2 \,\mathrm{I} \Phi -\mathrm{I} \phi -\mathrm{I} t}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}+\frac{\mathrm{I} \left(8 A^{2} \Omega^{4}-16 A^{2} \Omega^{2}+4 F^{2} \Omega^{2}-\Omega^{4}+8 A^{2}+2 \Omega^{2}-1\right) F \Omega  \,{\mathrm e}^{-\mathrm{I} \Omega  t -\mathrm{I} \Phi}}{\left(\Omega -1\right)^{4} \left(\Omega +1\right)^{4}}+\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{3 \,\mathrm{I} \Omega  t +3 \,\mathrm{I} \Phi}}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(3 \Omega -1\right) \left(3 \Omega +1\right)}+\frac{\mathrm{I} A^{3} {\mathrm e}^{-3 \,\mathrm{I} \phi -3 \,\mathrm{I} t}}{6}+\frac{4 \,\mathrm{I} \Omega  F \,A^{2} {\mathrm e}^{\mathrm{I} \Omega  t +\mathrm{I} \Phi +2 \,\mathrm{I} \phi +2 \,\mathrm{I} t}}{\left(\Omega -1\right) \left(\Omega +1\right)^{2} \left(\Omega +3\right)}$$

In [9]:
saecular := A*exp(I*phi) + e*coeff( firstcorrection, exp(I*t)) 
 + e*coeff( firstcorrection, exp(I*t+I*phi) )*exp(I*phi);

$$A \,{\mathrm e}^{\mathrm{I} \phi}-\frac{\varepsilon  A t \left(4 A^{2} \Omega^{4}-8 A^{2} \Omega^{2}+8 F^{2} \Omega^{2}-\Omega^{4}+4 A^{2}+2 \Omega^{2}-1\right) {\mathrm e}^{\mathrm{I} \phi}}{2 \left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}$$

In [10]:
yA := saecular/(A*exp(I*phi));

$$\frac{A \,{\mathrm e}^{\mathrm{I} \phi}-\frac{\varepsilon  A t \left(4 A^{2} \Omega^{4}-8 A^{2} \Omega^{2}+8 F^{2} \Omega^{2}-\Omega^{4}+4 A^{2}+2 \Omega^{2}-1\right) {\mathrm e}^{\mathrm{I} \phi}}{2 \left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}}{A \,{\mathrm e}^{\mathrm{I} \phi}}$$

In [11]:
yA := collect( yA, e, m->collect(m,[F,A], factor));

$$1+\left(-\frac{4 t \,\Omega^{2} F^{2}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 t \,A^{2}+\frac{t}{2}\right) \varepsilon$$

In [12]:
assumption_list := F>0, e>0, phi::real, Phi::real, t>0, A>0, Omega::real;

$$0<F,0<\varepsilon,\phi ::\mathit{real},\Phi ::\mathit{real},0<t,0<A,\Omega ::\mathit{real}$$

In [13]:
rhsR := simplify( Re( convert(series( diff(yA,t)/yA,e,2),polynom) ) ) assuming assumption_list;

$$\left(-\frac{4 \Omega^{2} F^{2}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 A^{2}+\frac{1}{2}\right) \varepsilon$$

In [14]:
rhsR := collect( eval(rhsR,A=R), e, m->combine(m,trig) );

$$\left(-\frac{4 \Omega^{2} F^{2}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 R^{2}+\frac{1}{2}\right) \varepsilon$$

In [15]:
rhsR := collect( rhsR, [e,F,R], distributed, factor ); 

$$\frac{\varepsilon}{2}-\frac{4 \Omega^{2} F^{2} \varepsilon}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 R^{2} \varepsilon$$

In [16]:
slowRderiv := R*convert( rhsR, polynom );

$$R \left(\frac{\varepsilon}{2}-\frac{4 \Omega^{2} F^{2} \varepsilon}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 R^{2} \varepsilon \right)$$

In [17]:
`diff/R` := codegen[makeproc](eval(slowRderiv*'diff'('expr', 'var'), [R = R(expr), phi = theta(expr),t=expr]), parameters = [expr, var]);

proc (expr, var) R(expr)*(1/2*varepsilon-4*Omega^2/(Omega-1)^2/(Omega+1)^2*F^2*varepsilon-2*R(expr)^2*varepsilon)*diff(expr,var) end proc

In [18]:
diff(R(t),t);

$$R \left(t \right) \left(\frac{\varepsilon}{2}-\frac{4 \Omega^{2} F^{2} \varepsilon}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 R \left(t \right)^{2} \varepsilon \right)$$

In [19]:
rhstheta := simplify( Im( convert(series( diff(yA,t)/yA,e,2),polynom) ) ) assuming assumption_list;

$$0$$

In [20]:
rhstheta := combine(rhstheta,trig);

$$0$$

In [21]:
`diff/theta` := codegen[makeproc](eval(rhstheta*'diff'('expr', 'var'), [R = R(expr), phi = theta(expr), t=expr]), parameters = [expr, var])

proc (expr, var) 0 end proc

In [22]:
diff(theta(t),t);

$$0$$

Now we re-do the computation (we should have checked the residual up there to see if it was $O(\e^2)$, but we will do it once we have recomputed).

In [23]:
yr0 := collect( combine( convert( eval( simplersol, [A=R(t), phi=theta(t)]), trig ), trig ), [R(t),F], normal ) assuming assumption_list;

$$2 \cos \left(\theta \left(t \right)+t \right) R \left(t \right)-\frac{2 F \cos \left(\Omega  t +\Phi \right)}{\Omega^{2}-1}$$

In [24]:
residual := eval( WeaklyNonlinear, y(t)=yr0 ):

In [25]:
residualSeries := map( combine, series( convert(residual,trig), e, 2 ), trig );

$$-\frac{-8 R \left(t \right)^{3} \Omega^{6} \sin \left(3 \theta \left(t \right)+3 t \right)-48 F R \left(t \right)^{2} \sin \left(\Omega  t +\Phi \right) \Omega^{5}+24 F R \left(t \right)^{2} \Omega^{5} \sin \left(\Omega  t -2 \theta \left(t \right)+\Phi -2 t \right)+24 F R \left(t \right)^{2} \Omega^{5} \sin \left(\Omega  t +2 \theta \left(t \right)+\Phi +2 t \right)-24 F^{2} R \left(t \right) \Omega^{4} \sin \left(2 \Omega  t +\theta \left(t \right)+2 \Phi +t \right)+24 F^{2} R \left(t \right) \Omega^{4} \sin \left(2 \Omega  t -\theta \left(t \right)+2 \Phi -t \right)+24 R \left(t \right)^{3} \Omega^{4} \sin \left(3 \theta \left(t \right)+3 t \right)-24 F^{3} \sin \left(\Omega  t +\Phi \right) \Omega^{3}+8 F^{3} \Omega^{3} \sin \left(3 \Omega  t +3 \Phi \right)+96 F R \left(t \right)^{2} \sin \left(\Omega  t +\Phi \right) \Omega^{3}-48 F R \left(t \right)^{2} \Omega^{3} \sin \left(\Omega  t -2 \theta \left(t \right)+\Phi -2 t \right)-48 F R \left(t \right)^{2} \Omega^{3} \sin \left(\Omega  t +2 \theta \left(t \right)+\Phi +2 t \right)+6 F \sin \left(\Omega  t +\Phi \right) \Omega^{5}+24 F^{2} R \left(t \right) \Omega^{2} \sin \left(2 \Omega  t +\theta \left(t \right)+2 \Phi +t \right)-24 F^{2} R \left(t \right) \Omega^{2} \sin \left(2 \Omega  t -\theta \left(t \right)+2 \Phi -t \right)-24 R \left(t \right)^{3} \Omega^{2} \sin \left(3 \theta \left(t \right)+3 t \right)-48 F R \left(t \right)^{2} \sin \left(\Omega  t +\Phi \right) \Omega +24 F R \left(t \right)^{2} \Omega  \sin \left(\Omega  t -2 \theta \left(t \right)+\Phi -2 t \right)+24 F R \left(t \right)^{2} \Omega  \sin \left(\Omega  t +2 \theta \left(t \right)+\Phi +2 t \right)-12 F \sin \left(\Omega  t +\Phi \right) \Omega^{3}+8 R \left(t \right)^{3} \sin \left(3 \theta \left(t \right)+3 t \right)+6 F \Omega  \sin \left(\Omega  t +\Phi \right)}{3 \Omega^{6}-9 \Omega^{4}+9 \Omega^{2}-3} \varepsilon +\mathrm{O}\left(\varepsilon^{2}\right)$$

To solve the first-order correction, we have to hide the dependence of R and theta on t from dsolve.  Because their derivatives are $O(\e)$, they are constant for the purpose of perturbative correction.

In [26]:
rhsc := collect( eval(coeff(residualSeries,e,1),[R(t)=RR,theta(t)=THETA]), [sin,cos], m->collect(m,[RR,F],factor) );

$$\frac{8 \mathit{RR}^{3} \sin \left(3 \mathit{THETA} +3 t \right)}{3}+\left(\frac{16 \Omega  F \,\mathit{RR}^{2}}{\left(\Omega -1\right) \left(\Omega +1\right)}+\frac{8 \Omega^{3} F^{3}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3}}-\frac{2 \Omega  F}{\left(\Omega -1\right) \left(\Omega +1\right)}\right) \sin \left(\Omega  t +\Phi \right)-\frac{8 \Omega^{3} F^{3} \sin \left(3 \Omega  t +3 \Phi \right)}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3}}-\frac{8 \Omega  F \,\mathit{RR}^{2} \sin \left(\Omega  t +\Phi -2 \mathit{THETA} -2 t \right)}{\left(\Omega -1\right) \left(\Omega +1\right)}-\frac{8 \Omega  F \,\mathit{RR}^{2} \sin \left(\Omega  t +\Phi +2 \mathit{THETA} +2 t \right)}{\left(\Omega -1\right) \left(\Omega +1\right)}-\frac{8 \Omega^{2} F^{2} \mathit{RR} \sin \left(2 \Omega  t +2 \Phi -\mathit{THETA} -t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}+\frac{8 \Omega^{2} F^{2} \mathit{RR} \sin \left(2 \Omega  t +2 \Phi +\mathit{THETA} +t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}$$

Maple's "dsolve" thinks the answer is too complicated to give to us unless we ask _really nicely_.  So we use our special-purpose solver.

In [27]:
correction :=SHOlveTerm( -rhsc, t, 1 );

$$-\frac{\mathrm{I} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-1\right)}+\frac{4 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(\Omega +2\right) t +\Phi +2 \mathit{THETA} \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}+4 \Omega +3\right)}+\frac{4 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(\Omega -2\right) t +\Phi -2 \mathit{THETA} \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-4 \Omega +3\right)}-\frac{\mathrm{I} \mathit{RR} \,F^{2} \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(2 \Omega +1\right) t +2 \Phi +\mathit{THETA} \right)}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}+\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(\Omega^{2}-1\right)}-\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(\Omega^{2}-1\right)}-\frac{4 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(-\Omega -2\right) t -\Phi -2 \mathit{THETA} \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}+4 \Omega +3\right)}-\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(-3 \Omega  t -3 \Phi \right)}}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(9 \Omega^{2}-1\right)}+\frac{\mathrm{I} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-1\right)}+\frac{\mathrm{I} \mathit{RR} \,F^{2} \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(-2 \Omega -1\right) t -2 \Phi -\mathit{THETA} \right)}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}+\frac{\mathrm{I} \mathit{RR} \,F^{2} \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(2 \Omega -1\right) t +2 \Phi -\mathit{THETA} \right)}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}+\frac{8 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(-\Omega  t -\Phi \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-1\right)}+\frac{\mathrm{I} \mathit{RR}^{3} {\mathrm e}^{\mathrm{I} \left(-3 \mathit{THETA} -3 t \right)}}{6}-\frac{\mathrm{I} \mathit{RR} \,F^{2} \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(-2 \Omega +1\right) t -2 \Phi +\mathit{THETA} \right)}}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}+\frac{4 \,\mathrm{I} F^{3} \Omega^{3} {\mathrm e}^{\mathrm{I} \left(3 \Omega  t +3 \Phi \right)}}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(9 \Omega^{2}-1\right)}-\frac{8 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\Omega  t +\Phi \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-1\right)}-\frac{4 \,\mathrm{I} \mathit{RR}^{2} F \Omega  \,{\mathrm e}^{\mathrm{I} \left(\left(-\Omega +2\right) t -\Phi +2 \mathit{THETA} \right)}}{\left(\Omega -1\right) \left(\Omega +1\right) \left(\Omega^{2}-4 \Omega +3\right)}-\frac{\mathrm{I} \mathit{RR}^{3} {\mathrm e}^{\mathrm{I} \left(3 \mathit{THETA} +3 t \right)}}{6}$$

Some combination of simplification commands cleans up the answer to be intelligible.

In [28]:
better := collect( combine(convert(correction,trig),trig), [sin,cos], m->collect(m,[RR,F], factor));

$$\frac{\mathit{RR}^{3} \sin \left(3 \mathit{THETA} +3 t \right)}{3}+\left(\frac{16 \Omega  F \,\mathit{RR}^{2}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}+\frac{8 \Omega^{3} F^{3}}{\left(\Omega -1\right)^{4} \left(\Omega +1\right)^{4}}-\frac{2 \Omega  F}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}\right) \sin \left(\Omega  t +\Phi \right)-\frac{8 \Omega^{3} F^{3} \sin \left(3 \Omega  t +3 \Phi \right)}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(3 \Omega -1\right) \left(3 \Omega +1\right)}-\frac{8 \Omega  F \,\mathit{RR}^{2} \sin \left(\Omega  t +\Phi -2 \mathit{THETA} -2 t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right) \left(\Omega -3\right)}-\frac{8 \Omega  F \,\mathit{RR}^{2} \sin \left(\Omega  t +\Phi +2 \mathit{THETA} +2 t \right)}{\left(\Omega -1\right) \left(\Omega +1\right)^{2} \left(\Omega +3\right)}-\frac{2 \mathit{RR} \,F^{2} \Omega  \sin \left(2 \Omega  t +2 \Phi -\mathit{THETA} -t \right)}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}+\frac{2 \mathit{RR} \,F^{2} \Omega  \sin \left(2 \Omega  t +2 \Phi +\mathit{THETA} +t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}$$

Looking carefully at the terms in that correction, we see $3\Omega \pm 1$ in the denominator of some terms, so $\Omega = \pm 1/3$ will be a superharmonic resonance; we also see $\Omega \pm 3$ in the denominator of other terms, so $\Omega = \pm 3$ will be a subharmonic resonance.

In [29]:
yr1 := yr0 + e*eval(better,[RR=R(t),THETA=theta(t)]);

$$2 \cos \left(\theta \left(t \right)+t \right) R \left(t \right)-\frac{2 F \cos \left(\Omega  t +\Phi \right)}{\Omega^{2}-1}+\varepsilon  \left(\frac{R \left(t \right)^{3} \sin \left(3 \theta \left(t \right)+3 t \right)}{3}+\left(\frac{16 \Omega  F R \left(t \right)^{2}}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}+\frac{8 \Omega^{3} F^{3}}{\left(\Omega -1\right)^{4} \left(\Omega +1\right)^{4}}-\frac{2 \Omega  F}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}\right) \sin \left(\Omega  t +\Phi \right)-\frac{8 \Omega^{3} F^{3} \sin \left(3 \Omega  t +3 \Phi \right)}{3 \left(\Omega -1\right)^{3} \left(\Omega +1\right)^{3} \left(3 \Omega -1\right) \left(3 \Omega +1\right)}-\frac{8 \Omega  F R \left(t \right)^{2} \sin \left(\Omega  t -2 \theta \left(t \right)+\Phi -2 t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right) \left(\Omega -3\right)}-\frac{8 \Omega  F R \left(t \right)^{2} \sin \left(\Omega  t +2 \theta \left(t \right)+\Phi +2 t \right)}{\left(\Omega -1\right) \left(\Omega +1\right)^{2} \left(\Omega +3\right)}-\frac{2 R \left(t \right) F^{2} \Omega  \sin \left(2 \Omega  t -\theta \left(t \right)+2 \Phi -t \right)}{\left(\Omega -1\right)^{3} \left(\Omega +1\right)^{2}}+\frac{2 R \left(t \right) F^{2} \Omega  \sin \left(2 \Omega  t +\theta \left(t \right)+2 \Phi +t \right)}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{3}}\right)$$

The fact that _this_ final residual is $O(\e^2)$ proves that we have found a good solution.

In [30]:
finalresidual := eval(WeaklyNonlinear,y(t)=yr1):
map(LargeExpressions:-Veil[K], map(simplify, series(finalresidual, e, 3)));

kilobytes used=74829, alloc=23835, time=6.01

$$\frac{K_{1}}{2} \varepsilon^{2}+K_{2} \varepsilon^{3}$$

In [31]:
diff(R(t),t);

$$R \left(t \right) \left(\frac{\varepsilon}{2}-\frac{4 \Omega^{2} F^{2} \varepsilon}{\left(\Omega -1\right)^{2} \left(\Omega +1\right)^{2}}-2 R \left(t \right)^{2} \varepsilon \right)$$